# Bridge MCP Servers to A2A Protocol

This cookbook demonstrates how to expose MCP (Model Context Protocol) servers as A2A-compatible services, enabling seamless interoperability between different agent ecosystems.

## Why Bridge MCP to A2A?

- **Protocol Interoperability**: Make MCP tools available to Google's A2A ecosystem
- **Enterprise Integration**: Connect local tools with enterprise agent systems
- **Identity Support**: Leverage AGNTCY Identity for secure tool access
- **Zero Changes**: MCP servers work as-is without modifications

This tutorial assumes familiarity with both MCP and A2A. If you're new to these protocols:
- MCP: See [Creating an agent with MCP](./mcp_agent.ipynb)
- A2A: See [Serve an Agent with A2A](./serve_a2a.ipynb)

## Install Dependencies

You'll need both MCP and A2A support:

In [ ]:
%pip install 'any-agent[mcp,a2a]'

import nest_asyncio
nest_asyncio.apply()

In [ ]:
import os
from getpass import getpass

# This example uses Mistral models
if "MISTRAL_API_KEY" not in os.environ:
    print("MISTRAL_API_KEY not found in environment!")
    api_key = getpass("Please enter your MISTRAL_API_KEY: ")
    os.environ["MISTRAL_API_KEY"] = api_key
    print("MISTRAL_API_KEY set for this session!")
else:
    print("MISTRAL_API_KEY found in environment.")

## Basic Bridge: Single MCP Server

Let's start by bridging a single MCP server (time server) to A2A:

In [ ]:
from any_agent.config import MCPStdio
from any_agent.serving import (
    MCPToA2ABridgeConfig,
    serve_mcp_as_a2a_async,
)

# Configure MCP server
mcp_config = MCPStdio(
    command="uvx",
    args=["mcp-server-time", "--local-timezone=America/New_York"],
    tools=["get_current_time"],
)

# Configure bridge
bridge_config = MCPToA2ABridgeConfig(
    mcp_config=mcp_config,
    port=0,  # Use dynamic port
    endpoint="/time-bridge",
    server_name="time-server",
)

# Start the bridge
bridge_handle = await serve_mcp_as_a2a_async(mcp_config, bridge_config)
bridge_port = bridge_handle.port

print(f"✓ MCP time server bridged to A2A at: http://localhost:{bridge_port}/time-bridge")

## Use the Bridged MCP Tool via A2A

Now we can access the MCP tool through A2A protocol:

In [ ]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import a2a_tool_async

# Create an A2A tool from the bridged MCP server
time_tool = await a2a_tool_async(
    f"http://localhost:{bridge_port}/time-bridge",
    toolname="get_time_via_a2a",
    http_kwargs={"timeout": 30},
)

# Create an agent that uses the bridged tool
agent = await AnyAgent.create_async(
    "tinyagent",
    AgentConfig(
        model_id="mistral/mistral-small-latest",
        instructions="Use the available tools to answer questions about time.",
        tools=[time_tool],
    ),
)

# Use the MCP tool through A2A
result = await agent.run_async("What time is it in New York?")
print(f"Agent response: {result.final_output}")

## Advanced: Bridge with AGNTCY Identity

If you're using mcpd with AGNTCY Identity support, you can include identity information in the bridge configuration. While the identity won't be exposed through the A2A protocol (due to AgentCard limitations), it will be logged and can be used for internal tracking:

In [ ]:
import json
from pathlib import Path

# Check if identity exists (from mcpd)
identity_path = Path("~/.config/mcpd/identity/time-server.json").expanduser()
identity_id = None

if identity_path.exists():
    with open(identity_path) as f:
        identity_data = json.load(f)
        identity_id = identity_data.get("id")
    print(f"Found identity: {identity_id}")
else:
    print("No identity found - proceeding without identity")

# Create bridge with identity
bridge_config_with_id = MCPToA2ABridgeConfig(
    mcp_config=mcp_config,
    port=0,
    endpoint="/secure-time",
    server_name="time-server",
    identity_id=identity_id,  # Will be logged but not exposed via A2A
)

# The identity will be logged when the bridge starts
secure_bridge = await serve_mcp_as_a2a_async(mcp_config, bridge_config_with_id)
print(f"✓ Secure bridge started (check logs for identity verification)")

## Multi-Bridge: Orchestrating Multiple MCP Tools

Let's bridge multiple MCP servers and orchestrate them:

In [ ]:
# Bridge multiple MCP servers
bridges = []
tools = []

# Time server
time_bridge = await serve_mcp_as_a2a_async(
    MCPStdio(
        command="uvx",
        args=["mcp-server-time"],
        tools=["get_current_time"],
    ),
    MCPToA2ABridgeConfig(
        mcp_config=None,  # Will be set by serve function
        port=0,
        endpoint="/time",
        server_name="time",
    ),
)
bridges.append(time_bridge)
tools.append(
    await a2a_tool_async(
        f"http://localhost:{time_bridge.port}/time",
        toolname="time_tool",
    )
)

# Add more MCP servers here (filesystem, github, etc.)
# Example: filesystem_bridge = await serve_mcp_as_a2a_async(...)

print(f"✓ Bridged {len(bridges)} MCP servers to A2A")

# Create orchestrator agent with all bridged tools
orchestrator = await AnyAgent.create_async(
    "tinyagent",
    AgentConfig(
        model_id="mistral/mistral-small-latest",
        name="orchestrator",
        instructions="Coordinate multiple tools to answer complex questions.",
        tools=tools,
    ),
)

# Complex query using multiple tools
result = await orchestrator.run_async(
    "What's the current time? Format it nicely for a report."
)
print(f"\nOrchestrator response:\n{result.final_output}")

## Direct Tool Invocation

The bridge also supports direct tool invocation using a simple protocol:

In [ ]:
import httpx
from a2a.client import A2AClient, A2ACardResolver
from a2a.types import (
    Message,
    MessageSendParams,
    Part,
    Role,
    SendMessageRequest,
    TextPart,
)
from uuid import uuid4

# Get the agent card
bridge_url = f"http://localhost:{bridge_port}/time-bridge"
async with httpx.AsyncClient() as client:
    resolver = A2ACardResolver(httpx_client=client, base_url=bridge_url)
    agent_card = await resolver.get_agent_card()
    print(f"Agent: {agent_card.name}")
    print(f"Skills: {[s.name for s in agent_card.skills]}")

# Direct tool call
async with httpx.AsyncClient() as client:
    a2a_client = A2AClient(httpx_client=client, agent_card=agent_card)
    
    # For single tool bridges, just send the input
    request = SendMessageRequest(
        id=str(uuid4()),
        params=MessageSendParams(
            message=Message(
                role=Role.user,
                parts=[Part(root=TextPart(text="current time"))],
                message_id=str(uuid4()),
            )
        ),
    )
    
    response = await a2a_client.send_message(request)
    print(f"\nDirect response: {response}")

## Cleanup

Don't forget to clean up the bridges:

In [ ]:
# Shutdown all bridges
for bridge in bridges:
    await bridge.shutdown()

await bridge_handle.shutdown()
await secure_bridge.shutdown()

print("✓ All bridges shut down")

## Next Steps

1. **Production Deployment**: Deploy bridges as microservices
2. **Service Discovery**: Register bridges with AGNTCY Directory
3. **Authentication**: Add proper auth between bridges and agents
4. **Monitoring**: Add observability for bridge operations

For more information:
- [MCP Documentation](https://modelcontextprotocol.io/)
- [A2A Protocol](https://github.com/google/a2a)
- [AGNTCY Identity](https://github.com/agntcy/identity)